In [21]:
import pandas as pd
import numpy as np
import os

data_path = r"E:\DeepLearning\datasets"
train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
images_path = os.path.join(data_path, "images")

In [22]:
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [23]:
train_df.shape

(1821, 5)

In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1821 entries, 0 to 1820
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image_id           1821 non-null   object
 1   healthy            1821 non-null   int64 
 2   multiple_diseases  1821 non-null   int64 
 3   rust               1821 non-null   int64 
 4   scab               1821 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 71.3+ KB


In [25]:
train_df.isnull().sum()

image_id             0
healthy              0
multiple_diseases    0
rust                 0
scab                 0
dtype: int64

In [26]:
train_df['label'] = train_df[['healthy', 'multiple_diseases', 'rust', 'scab']].idxmax(axis=1)
train_df[['image_id', 'label']].head()

,image_id,label
0,Train_0,scab
1,Train_1,multiple_diseases
2,Train_2,healthy
3,Train_3,rust
4,Train_4,healthy


In [27]:
train_df.columns

Index(['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab', 'label'], dtype='object')

# Scikit Learn

### MLP

In [28]:
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

labels = train_df["label"].values
le = LabelEncoder()
y = le.fit_transform(labels)

In [29]:
# Đọc và resize ảnh, flatten thành vector
X = []
for img_id in train_df["image_id"]:
    img_path = os.path.join(images_path, img_id + ".jpg")
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))   # resize nhỏ cho nhanh
    X.append(img.flatten())

X = np.array(X)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [31]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64), 
    activation='relu',
    max_iter=20, 
    random_state=42
)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

print("MLP Classifier Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

MLP Classifier Report:
                   precision    recall  f1-score   support

          healthy       0.41      0.31      0.35       100
multiple_diseases       0.00      0.00      0.00        18
             rust       0.36      0.86      0.51       120
             scab       1.00      0.01      0.02       127

         accuracy                           0.37       365
        macro avg       0.44      0.29      0.22       365
     weighted avg       0.58      0.37      0.27       365



C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


### BernoulliRBM

In [32]:
from sklearn.neural_network import BernoulliRBM
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

rbm = BernoulliRBM(n_components=64, learning_rate=0.01, n_iter=10, random_state=42)
logistic = LogisticRegression(max_iter=1000)

rbm_clf = Pipeline(steps=[("rbm", rbm), ("logistic", logistic)])

rbm_clf.fit(X_train, y_train)
y_pred_rbm = rbm_clf.predict(X_test)

print("RBM + Logistic Regression Report:")
print(classification_report(y_test, y_pred_rbm, target_names=le.classes_))

RBM + Logistic Regression Report:
                   precision    recall  f1-score   support

          healthy       0.00      0.00      0.00       100
multiple_diseases       0.00      0.00      0.00        18
             rust       0.33      1.00      0.49       120
             scab       0.00      0.00      0.00       127

         accuracy                           0.33       365
        macro avg       0.08      0.25      0.12       365
     weighted avg       0.11      0.33      0.16       365



C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape

#### Overfit

In [33]:
mlp = MLPClassifier(
    hidden_layer_sizes=(64,),  # giảm số neuron
    activation='relu',
    max_iter=100,
    random_state=42
)

In [36]:
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

from sklearn.metrics import classification_report
print("MLP Classifier Report (sau giảm overfit):")
print(classification_report(y_test, y_pred, target_names=le.classes_))


MLP Classifier Report (sau giảm overfit):
                   precision    recall  f1-score   support

          healthy       0.00      0.00      0.00       100
multiple_diseases       0.00      0.00      0.00        18
             rust       0.33      1.00      0.49       120
             scab       0.00      0.00      0.00       127

         accuracy                           0.33       365
        macro avg       0.08      0.25      0.12       365
     weighted avg       0.11      0.33      0.16       365



C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.p

#### Drop out

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Giả sử X đã flatten và normalize [0,1], y đã encode
y_cat = to_categorical(y, num_classes=len(le.classes_))

# MLP với Dropout cơ bản
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y_cat, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3132 - loss: 1.4511 - val_accuracy: 0.3507 - val_loss: 1.2619
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3194 - loss: 1.2919 - val_accuracy: 0.3014 - val_loss: 1.2717
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3207 - loss: 1.2737 - val_accuracy: 0.3288 - val_loss: 1.2546
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3372 - loss: 1.2811 - val_accuracy: 0.3425 - val_loss: 1.2632
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3565 - loss: 1.2606 - val_accuracy: 0.3425 - val_loss: 1.2905
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3475 - loss: 1.2707 - val_accuracy: 0.3425 - val_loss: 1.2686
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3441 - loss: 1.2511 - val_accuracy: 0.3425 - val_loss: 1.2593
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3317 - loss: 1.2445 - val_accuracy: 0.3425 - val_loss:

#### Multi layer (chọn tầng hiệu quả)

In [40]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Giả sử X, y đã chuẩn bị sẵn
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [41]:
#1 tầng
mlp1 = MLPClassifier(hidden_layer_sizes=(64,), max_iter=200, random_state=42)
mlp1.fit(X_train, y_train)
y_pred1 = mlp1.predict(X_val)
print("MLP 1 hidden layer:")
print(classification_report(y_val, y_pred1, target_names=le.classes_))

MLP 1 hidden layer:
                   precision    recall  f1-score   support

          healthy       0.38      0.51      0.44       103
multiple_diseases       0.06      0.06      0.06        18
             rust       0.40      0.26      0.32       125
             scab       0.33      0.34      0.34       119

         accuracy                           0.35       365
        macro avg       0.29      0.29      0.29       365
     weighted avg       0.35      0.35      0.34       365



C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [42]:
#2 tầng
mlp2 = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=200, random_state=42)
mlp2.fit(X_train, y_train)
y_pred2 = mlp2.predict(X_val)
print("MLP 2 hidden layers:")
print(classification_report(y_val, y_pred2, target_names=le.classes_))

MLP 2 hidden layers:
                   precision    recall  f1-score   support

          healthy       0.31      0.72      0.43       103
multiple_diseases       0.00      0.00      0.00        18
             rust       0.40      0.05      0.09       125
             scab       0.31      0.29      0.30       119

         accuracy                           0.31       365
        macro avg       0.26      0.26      0.20       365
     weighted avg       0.33      0.31      0.25       365



In [43]:
mlp3 = MLPClassifier(hidden_layer_sizes=(128,64,32), max_iter=200, random_state=42)
mlp3.fit(X_train, y_train)
y_pred3 = mlp3.predict(X_val)
print("MLP 3 hidden layers:")
print(classification_report(y_val, y_pred3, target_names=le.classes_))

MLP 3 hidden layers:
                   precision    recall  f1-score   support

          healthy       0.64      0.09      0.15       103
multiple_diseases       0.00      0.00      0.00        18
             rust       0.35      0.07      0.12       125
             scab       0.34      0.92      0.49       119

         accuracy                           0.35       365
        macro avg       0.33      0.27      0.19       365
     weighted avg       0.41      0.35      0.25       365



# Tensor

### CNN

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# X reshape về 2D cho CNN
X_cnn = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # dùng lại X đã chuẩn hóa
y_cat = to_categorical(y, num_classes=len(le.classes_))

In [45]:
cnn_model = Sequential([
    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model.fit(X_cnn, y_cat, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20


C:\Users\hilda\anaconda3\envs\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.3338 - loss: 1.2915 - val_accuracy: 0.3425 - val_loss: 1.2609
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3393 - loss: 1.2473 - val_accuracy: 0.4247 - val_loss: 1.2598
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.3709 - loss: 1.2284 - val_accuracy: 0.3342 - val_loss: 1.2527
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3942 - loss: 1.2138 - val_accuracy: 0.4192 - val_loss: 1.2249
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.4238 - loss: 1.1999 - val_accuracy: 0.3479 - val_loss: 1.2419
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.4471 - loss: 1.1763 - val_accuracy: 0.4466 - val_loss: 1.1871
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.4746 - loss: 1.1402 - val_accuracy: 0.4493 - val_loss: 1.1921
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5220 - loss: 1.0902 - val_accuracy: 0.4575 - val_loss: 1.

### MLP

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

mlp_tensor = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

mlp_tensor.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mlp_tensor.summary()

mlp_tensor.fit(X, y_cat, epochs=20, batch_size=32, validation_split=0.2)

y_pred = mlp_tensor.predict(X)
y_class = y_pred.argmax(axis=1)   

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_26 (Dense)                     │ (None, 128)                 │         524,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 532,932 (2.03 MB)

 Trainable params: 532,932 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3290 - loss: 1.3796 - val_accuracy: 0.3068 - val_loss: 1.2917
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3496 - loss: 1.2621 - val_accuracy: 0.3644 - val_loss: 1.2664
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3482 - loss: 1.2745 - val_accuracy: 0.3699 - val_loss: 1.2423
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3949 - loss: 1.2199 - val_accuracy: 0.3315 - val_loss: 1.3214
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4011 - loss: 1.2140 - val_accuracy: 0.3534 - val_loss: 1.2950
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4299 - loss: 1.1986 - val_accuracy: 0.3589 - val_loss: 1.2667
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4004 - loss: 1.2112 - val_accuracy: 0.3452 - val_loss: 1.2780
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4100 - loss: 1.1904 - val_accuracy: 0.3671 - val_loss:

#### Overfit

#### Drop out

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

mlp_tensor = Sequential([
    Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])


In [59]:
mlp_tensor.compile(optimizer='adam', 
                   loss='categorical_crossentropy', 
                   metrics=['accuracy'])


In [60]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

mlp_tensor.fit(X_scaled, y_cat, 
               epochs=50, 
               batch_size=32, 
               validation_split=0.2, 
               callbacks=[early_stop])


Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3104 - loss: 2.3234 - val_accuracy: 0.3425 - val_loss: 1.6441
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4155 - loss: 1.6538 - val_accuracy: 0.3123 - val_loss: 1.5154
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4615 - loss: 1.3022 - val_accuracy: 0.4110 - val_loss: 1.3408
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5364 - loss: 1.1310 - val_accuracy: 0.3836 - val_loss: 1.3465
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5515 - loss: 1.0739 - val_accuracy: 0.4521 - val_loss: 1.3197
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5460 - loss: 1.0646 - val_accuracy: 0.3808 - val_loss: 1.3458
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5955 - loss: 0.9364 - val_accuracy: 0.3890 - val_loss: 1.3354
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6229 - loss: 0.8781 - val_accuracy: 0.3973 - val_loss:

#### Multi layer (chọn tầng hiệu quả)

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model_1 = Sequential([
    Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])

model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_1 = model_1.fit(X_scaled, y_cat, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3558 - loss: 2.7390 - val_accuracy: 0.3863 - val_loss: 2.6382
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5247 - loss: 1.6542 - val_accuracy: 0.3479 - val_loss: 2.2455
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6250 - loss: 1.1400 - val_accuracy: 0.3890 - val_loss: 2.2221
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7081 - loss: 0.8617 - val_accuracy: 0.3534 - val_loss: 2.3086
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7534 - loss: 0.6702 - val_accuracy: 0.3753 - val_loss: 2.3258
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8049 - loss: 0.5486 - val_accuracy: 0.3562 - val_loss: 2.1885
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8578 - loss: 0.3962 - val_accuracy: 0.3726 - val_loss: 2.5322
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8626 - loss: 0.3619 - val_accuracy: 0.3671 - val_loss

In [62]:
model_2 = Sequential([
    Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])

model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_2 = model_2.fit(X_scaled, y_cat, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3159 - loss: 2.0250 - val_accuracy: 0.3699 - val_loss: 1.4204
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3990 - loss: 1.4619 - val_accuracy: 0.3507 - val_loss: 1.3543
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4396 - loss: 1.2573 - val_accuracy: 0.3397 - val_loss: 1.3611
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4766 - loss: 1.1602 - val_accuracy: 0.3644 - val_loss: 1.3077
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5247 - loss: 1.0697 - val_accuracy: 0.3699 - val_loss: 1.2926
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5900 - loss: 0.9521 - val_accuracy: 0.3781 - val_loss: 1.3499
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6092 - loss: 0.9240 - val_accuracy: 0.4055 - val_loss: 1.3311
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6332 - loss: 0.8865 - val_accuracy: 0.3753 - val_loss:

In [63]:
model_3 = Sequential([
    Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])

model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_3 = model_3.fit(X_scaled, y_cat, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3001 - loss: 2.0119 - val_accuracy: 0.3507 - val_loss: 1.3766
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3599 - loss: 1.5421 - val_accuracy: 0.3863 - val_loss: 1.3005
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3846 - loss: 1.3975 - val_accuracy: 0.3616 - val_loss: 1.2957
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3887 - loss: 1.3456 - val_accuracy: 0.3425 - val_loss: 1.2806
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4409 - loss: 1.2693 - val_accuracy: 0.3096 - val_loss: 1.2877
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4231 - loss: 1.2509 - val_accuracy: 0.3315 - val_loss: 1.2609
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4409 - loss: 1.2040 - val_accuracy: 0.3616 - val_loss: 1.2733
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4526 - loss: 1.1915 - val_accuracy: 0.3808 - val_loss: